In [1]:
# TASK 1: Analysis of step-one performance retrieval.
# First thing, start easy: build two-indexes, evaluate them top-m retrievals
# Hit-once and Hit-all. Then see other metrics like precision & recall + Roni's analysis.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# cleaning sample_data directory
!rm -r sample_data

In [3]:
!ls drive/My\ Drive/touche-2022-prototyping

dataset-prep-and-retrieval-diagnostic-analysis.ipynb
indexes
initial-retrieval-metric-analysis.ipynb
merged_documents
missed_full_1000.csv
missed_full_1375.csv
missed_full_1500.csv
missed_full_250.csv
missed_full_500.csv
missed_full_786.csv
mon-duo-retrieval-prototyping-and-analysis.ipynb
topics-task2-51-100.xml
topics-task-2.xml
touche2020-task2-relevance-withbaseline.qrels
touche_complete_topics.csv
touche_ground_truth.csv
touche_results_2021.csv
touche-task2-51-100-relevance.qrels
touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl


In [ ]:
# One Time Exection.
# extracting files and converting them to 'tar.gz' files.
# !gzip -d drive/My\ Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl.gz
# !gzip -d drive/My\ Drive/touche-2022-prototyping/touche-task2-passages-version-002.jsonl.gz

In [4]:
# installing linux related stuff for pyserini
# removing error "ImportError: No module named '_swigfaiss"
# Reference Link: https://github.com/facebookresearch/faiss/issues/821
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (369 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty

In [5]:
# installing important packages for analyzing the code.
!pip install jsonlines
!pip install pyserini
!pip install faiss

     |████████████████████████████████| 84.6 MB 149 kB/s 
     |████████████████████████████████| 13.5 MB 21.7 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 5.2 MB 48.7 MB/s 
     |████████████████████████████████| 4.0 MB 45.4 MB/s 
     |████████████████████████████████| 1.2 MB 67.1 MB/s 
     |████████████████████████████████| 2.0 MB 53.8 MB/s 
     |████████████████████████████████| 6.0 MB 42.7 MB/s 
     |████████████████████████████████| 188 kB 53.5 MB/s 
     |████████████████████████████████| 653 kB 60.4 MB/s 
     |████████████████████████████████| 10.1 MB 45.0 MB/s 
     |████████████████████████████████| 181 kB 56.7 MB/s 
     |████████████████████████████████| 451 kB 60.3 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████████████████████████| 77 kB 7.4 MB/s 
     |████████████████████████████████| 6.5 MB 34.3 MB/s 
     |██████████

In [6]:
# import statements
import pandas as pd
import jsonlines
from xml.dom import minidom
import numpy as np

In [ ]:
# original corpus
# corpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002.jsonl'))

In [ ]:
# query expansion corpus
qcorpus_list = list(jsonlines.open('drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl'))

In [ ]:
# printing demo query
qcorpus_list[0]

{'chatNoirUrl': 'https://chatnoir.eu/cache?uuid=25e04d49-8df7-58c9-8fae-c5bd54a070ba&index=cw12&raw&plain',
 'contents': "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>",
 'id': 'clueweb12-0000tw-00-14115___1'

In [ ]:
!mkdir sample_collection_jsonl
!mkdir indexes
!mkdir indexes/sample_collection_jsonl

In [ ]:
output = jsonlines.open('sample_collection_jsonl/documents.jsonl', 'w')
for vals in qcorpus_list:
    output.write({
        'id': vals['id'],
        'contents': vals['contents']
    })

In [ ]:
!head -10 sample_collection_jsonl/documents.jsonl

{"id": "clueweb12-0000tw-00-14115___1", "contents": "Do Asian-Americans Face Bias in Admissions at Elite Colleges? - NYTimes.com Home Page Today's Paper Video Most Popular Times Topics Search All NYTimes.com Education World U.S. Politics Education Bay Area Chicago Texas N.Y. / Region Business Technology Science Health Sports Opinion Arts Style Travel Jobs Real Estate Autos February 8, 2012, 1:43 pm Do Asian-Americans Face Bias in Admissions at Elite Colleges? By DANIEL E. SLOTNIK 6:08 p.m. | Updated A statement from Princeton was added to the story. The Department of Education’s Office for Civil Rights is examining complaints thatHarvard and Princeton have discriminated against Asian-American undergraduate applicants, highlighting a concern of many Asian-American parents. The inquiry was first reported by Bloomberg News.<query> do asian americans face bias in admissions</query>"}
{"id": "clueweb12-0000tw-00-14115___10", "contents": "By insisting on upholding affirmative action in colle

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/sample_collection_jsonl \
  --index /content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index \
  --generator DefaultLuceneDocumentGenerator \
  --threads 4 \
  --storePositions --storeDocvectors --storeRaw

2022-04-02 13:00:33,191 INFO  [main] index.IndexCollection (IndexCollection.java:643) - Setting log level to INFO
2022-04-02 13:00:33,198 INFO  [main] index.IndexCollection (IndexCollection.java:646) - Starting indexer...
2022-04-02 13:00:33,198 INFO  [main] index.IndexCollection (IndexCollection.java:647) - ============ Loading Parameters ============
2022-04-02 13:00:33,199 INFO  [main] index.IndexCollection (IndexCollection.java:648) - DocumentCollection path: /content/sample_collection_jsonl
2022-04-02 13:00:33,200 INFO  [main] index.IndexCollection (IndexCollection.java:649) - CollectionClass: JsonCollection
2022-04-02 13:00:33,205 INFO  [main] index.IndexCollection (IndexCollection.java:650) - Generator: DefaultLuceneDocumentGenerator
2022-04-02 13:00:33,206 INFO  [main] index.IndexCollection (IndexCollection.java:651) - Threads: 4
2022-04-02 13:00:33,206 INFO  [main] index.IndexCollection (IndexCollection.java:652) - Language: en
2022-04-02 13:00:33,207 INFO  [main] index.IndexC

In [ ]:
!ls /content/indexes/sample_collection_jsonl

_0.fdm	_0_Lucene80_0.dvd  _0_Lucene84_0.tim  _0.nvm  _0.tvx
_0.fdt	_0_Lucene80_0.dvm  _0_Lucene84_0.tip  _0.si   segments_1
_0.fdx	_0_Lucene84_0.doc  _0_Lucene84_0.tmd  _0.tvd  write.lock
_0.fnm	_0_Lucene84_0.pos  _0.nvd	      _0.tvm


In [7]:
from pyserini.search.lucene import LuceneSearcher
# searcher = LuceneSearcher('indexes/sample_collection_jsonl')
searcher = LuceneSearcher('/content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index')
hits = searcher.search('Who is stronger, Hulk or Superman?', k=12)
# Value of K determines the number of documents that can be returned.
print('Number of document matches from the index: '+str(len(hits)))
# Attempted tuning BM25 and RM3, nothing has changed the document ranking below.
for i in range(len(hits)):
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')

Number of document matches from the index: 12
1 clueweb12-1400wb-06-30596___3 16.55650
2 clueweb12-0605wb-29-30060___11 15.76430
3 clueweb12-1709wb-12-07226___7 15.69540
4 clueweb12-0605wb-29-30060___2 15.21990
5 clueweb12-0904wb-33-14589___7 15.14270
6 clueweb12-1400wb-06-30596___4 15.02100
7 clueweb12-1411wb-25-14634___8 14.83110
8 clueweb12-0605wb-29-30060___10 14.75490
9 clueweb12-1014wb-84-07457___12 14.68790
10 clueweb12-1400tw-67-08199___28 14.26160
11 clueweb12-0306wb-63-29079___5 14.16490
12 clueweb12-0605wb-57-17257___8 13.69220


In [8]:
# Grab the raw text:
print(hits[0].raw)
print('\n')
# Grab the raw Lucene Document:
print(hits[0].lucene_document)

{
  "id" : "clueweb12-1400wb-06-30596___3",
  "contents" : "There is a limit to how angry someone gets...even the Hulk and even at his angriest he wouldn't be as strong as Superman. No one really knows how strong the man of steel is and perhaps neither does he! Many wrongly think that just because Hulk is bigger and looks stronger than it means he is stronger. Superman can move planets...need I say more! Others believe Hulk would win because they're likely Marvel fan boys. Superman has defeated powerful opponents like Darkseid,Mongrel,Parasite,Doomsday,and General Zod;several of whom could likely take Hulk! Superman has more varied powers and because he's smarter can be more resourceful in using them. Speaking of intelligence,how the hell can anyone think that Superman isn't smarter than a human,even one as brilliant as Dr.Bruce Banner. Superman/Kal-El is not only the son of Jor-El,Krypton's most brilliant scientist,but he's outsmarted Lex Luthor(one of the smartest people on earth)and

In [10]:
# topics_2021 = parse_xml("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-51-100.xml")
# topics_2021.head()
touche_topics = pd.read_csv("/content/drive/MyDrive/touche-2022-prototyping/touche_complete_topics.csv")
touche_topics.head() 

,Number,Title
0,1,\nWhat is the difference between sex and love?\n
1,2,"\nWhich is better, a laptop or a desktop?\n"
2,3,"\nWhich is better, Canon or Nikon?\n"
3,4,\nWhat are the best dish detergents?\n
4,5,\nWhat are the best cities to live in?\n


In [74]:
# Run loop for topics_2021 data, store retrieved in order of appearance m=275
solution_dict = {} # topic_id, corresponding_document list.
for id_, q_ in zip(touche_topics['Number'], touche_topics['Title']):
    hits = searcher.search(q_.strip(), k=1500)
    d_list = []
    for h_ in hits:
        d_= h_.docid.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)

    solution_dict[id_] = d_list

In [12]:
searcher_opt = LuceneSearcher('/content/drive/MyDrive/touche-2022-prototyping/indexes/baseline_index')
searcher_opt.set_bm25(1.2, 0.68)
hits = searcher_opt.search('Who is stronger, Hulk or Superman?', k=12)
# Value of K determines the number of documents that can be returned.
print('Number of document matches from the index: '+str(len(hits)))
for i in range(len(hits)):
    print(f'{i+1} {hits[i].docid} {hits[i].score:.5f}')

Number of document matches from the index: 12
1 clueweb12-1400wb-06-30596___3 15.37180
2 clueweb12-1709wb-12-07226___7 14.85930
3 clueweb12-0605wb-29-30060___11 14.84560
4 clueweb12-0605wb-29-30060___10 13.93880
5 clueweb12-0605wb-29-30060___2 13.85450
6 clueweb12-1400wb-06-30596___4 13.81460
7 clueweb12-0904wb-33-14589___7 13.78280
8 clueweb12-1411wb-25-14634___8 13.60230
9 clueweb12-1014wb-84-07457___12 13.35340
10 clueweb12-1400tw-67-08199___28 13.04680
11 clueweb12-0306wb-63-29079___5 12.96960
12 clueweb12-0605wb-57-17257___8 12.88170


In [75]:
# Run loop for topics_2021 data, store retrieved in order of appearance m=275
solution_dict_opt = {} # topic_id, corresponding_document list.
for id_, q_ in zip(touche_topics['Number'], touche_topics['Title']):
    hits = searcher_opt.search(q_.strip(), k=1500)
    d_list = []
    for h_ in hits:
        d_= h_.docid.split('___')[0]
        if d_ not in d_list:
            d_list.append(d_)

    solution_dict_opt[id_] = d_list

In [14]:
# creating new dataframe formatted file which only has the matching doc-ids with qrel file.
jsonObj = pd.read_json(path_or_buf='drive/My Drive/touche-2022-prototyping/touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl', lines=True)
jsonObj["old_id"]=jsonObj["id"].str.split("___",expand=True)[0]
documents_2021=jsonObj.groupby(by="old_id")["contents"].apply(" ".join).reset_index()
del jsonObj
documents_2021.head()

,old_id,contents
0,clueweb12-0000tw-00-14115,Do Asian-Americans Face Bias in Admissions at ...
1,clueweb12-0000tw-00-15738,Big Data Analytics a Key Enabler for Social CR...
2,clueweb12-0000tw-00-17905,Best Bets: Fern brings magic to Milford - Delm...
3,clueweb12-0000tw-01-02230,Python Data Analysis Library — pandas: Python ...
4,clueweb12-0000tw-01-15084,MILLARWORLD WEEK: Mark Millar’s Master Plan | ...


In [15]:
new_rel_2021 = pd.read_csv('/content/drive/MyDrive/touche-2022-prototyping/touche_ground_truth.csv')

In [16]:
new_rel_2021.head()

,qid,no,doc,rel
0,1,0,clueweb12-0001wb-05-12311,0
1,1,0,clueweb12-1811wb-62-08424,1
2,1,0,clueweb12-1811wb-62-08423,1
3,1,0,clueweb12-1217wb-47-14048,0
4,1,0,clueweb12-1811wb-62-08425,1


In [23]:
from collections import defaultdict
ground_truth_dict = defaultdict(list)
rel0_truth_dict = defaultdict(list)
rel1_truth_dict = defaultdict(list)
rel2_truth_dict = defaultdict(list)

for i_, d_, x_ in zip(new_rel_2021['qid'], new_rel_2021['doc'], new_rel_2021['rel']):
    i_ = int(i_)
    d_ = str(d_)    
    if int(x_) > 0:
        ground_truth_dict[i_].append(d_)
    if int(x_) == 0:
        rel0_truth_dict[i_].append(d_)
    if int(x_) == 1:
        rel1_truth_dict[i_].append(d_)
    if int(x_) == 2:
        rel2_truth_dict[i_].append(d_)

In [76]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 100
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.3
Average common ratio: 0.8764
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.08
Zero Relevance, Average common ratio: 0.7707
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.41
One Relevance, Average common ratio: 0.8602
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.8034


In [77]:
# the final dictionaries for basic metric evaluation and analysis.
# Average percentage common, Hit-once and Hit-all metric basic definition.
hit_one = 0
hit_all = 0
total = 100
per_comm_avg = 0

for id_i, doc_i in ground_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit_one += 1
            if doc_j.issuperset(doc_i):
                hit_all += 1
            per_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Hit one: {round(hit_one / total, 4)}')
print(f'Hit all: {round(hit_all / total, 4)}')
print(f'Average common ratio: {round(per_comm_avg / total, 4)}')

hit0_one = 0
hit0_all = 0
per0_comm_avg = 0

for id_i, doc_i in rel0_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit0_one += 1
            if doc_j.issuperset(doc_i):
                hit0_all += 1
            per0_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Zero Relevance, Hit one: {round(hit0_one / total, 4)}')
print(f'Zero Relevance, Hit all: {round(hit0_all / total, 4)}')
print(f'Zero Relevance, Average common ratio: {round(per0_comm_avg / total, 4)}')

hit1_one = 0
hit1_all = 0
per1_comm_avg = 0

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit1_one += 1
            if doc_j.issuperset(doc_i):
                hit1_all += 1
            per1_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'One Relevance, Hit one: {round(hit1_one / total, 4)}')
print(f'One Relevance, Hit all: {round(hit1_all / total, 4)}')
print(f'One Relevance, Average common ratio: {round(per1_comm_avg / total, 4)}')

hit2_one = 0
hit2_all = 0
per2_comm_avg = 0

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            if doc_j.intersection(doc_i):
                hit2_one += 1
            if doc_j.issuperset(doc_i):
                hit2_all += 1
            per2_comm_avg += len(doc_j.intersection(doc_i))/len(doc_i)
            break

print(f'Two Relevance, Hit one: {round(hit2_one / total, 4)}')
print(f'Two Relevance, Hit all: {round(hit2_all / total, 4)}')
print(f'Two Relevance, Average common ratio: {round(per2_comm_avg / total, 4)}')

Hit one: 1.0
Hit all: 0.3
Average common ratio: 0.8752
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.07
Zero Relevance, Average common ratio: 0.7668
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.4
One Relevance, Average common ratio: 0.8578
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.8034


In [ ]:
# Result summarization and A case for query expansion.
# Measuring on an average what's the document retrieval coverage.

1. k = 250, opt
Hit one: 1.0
Hit all: 0.15
Average common ratio: 0.7189
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.0
Zero Relevance, Average common ratio: 0.4287
One Relevance, Hit one: 0.96
One Relevance, Hit all: 0.18
One Relevance, Average common ratio: 0.6468
Two Relevance, Hit one: 0.86
Two Relevance, Hit all: 0.39
Two Relevance, Average common ratio: 0.7096

2. k = 500, opt
Hit one: 1.0
Hit all: 0.2
Average common ratio: 0.8113
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.0
Zero Relevance, Average common ratio: 0.5924
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.29
One Relevance, Average common ratio: 0.7618
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.47
Two Relevance, Average common ratio: 0.789

3. k = 786, opt
Hit one: 1.0
Hit all: 0.25
Average common ratio: 0.8444
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.04
Zero Relevance, Average common ratio: 0.6814
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.35
One Relevance, Average common ratio: 0.8145
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.48
Two Relevance, Average common ratio: 0.7959

4. k = 1000, opt
Hit one: 1.0
Hit all: 0.27
Average common ratio: 0.8581
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.06
Zero Relevance, Average common ratio: 0.713
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.37
One Relevance, Average common ratio: 0.8349
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.7973

5. k = 1200, opt
Hit one: 1.0
Hit all: 0.28
Average common ratio: 0.8665
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.07
Zero Relevance, Average common ratio: 0.7389
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.38
One Relevance, Average common ratio: 0.8425
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.8019

6. k = 1375, opt
Hit one: 1.0
Hit all: 0.29
Average common ratio: 0.8733
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.07
Zero Relevance, Average common ratio: 0.7569
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.39
One Relevance, Average common ratio: 0.851
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.8034

7. k = 1500, opt
Hit one: 1.0
Hit all: 0.3
Average common ratio: 0.8752
Zero Relevance, Hit one: 1.0
Zero Relevance, Hit all: 0.07
Zero Relevance, Average common ratio: 0.7668
One Relevance, Hit one: 0.99
One Relevance, Hit all: 0.4
One Relevance, Average common ratio: 0.8578
Two Relevance, Hit one: 0.9
Two Relevance, Hit all: 0.49
Two Relevance, Average common ratio: 0.8034

In [78]:
# preparing the missing document analysis, alongside the content
# for observing the limitations of the existing one-query retrieval approach.

tp_rel1_dict = {}
tp_rel2_dict = {}

for id_i, doc_i in rel1_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        # print(tp_rel1_dict)
        # print(id_i)
        if id_i == id_j :
            # print(tp_rel1_dict)
            # print(id_i)
            d_ = doc_i.difference(doc_j)
            tp_rel1_dict[id_i] = (list(d_))
            break

for id_i, doc_i in rel2_truth_dict.items():
    doc_i = set(doc_i)
    for id_j , doc_j in solution_dict_opt.items():
        doc_j = set(doc_j)
        if id_i == id_j:
            d_ = doc_i.difference(doc_j)
            tp_rel2_dict[id_i] = (list(d_))
            break

In [38]:
documents_2021 = documents_2021.astype({"old_id": str})
print(documents_2021.dtypes)
documents_2021_ids = documents_2021['old_id']
documents_2021_contents =  documents_2021['contents']
print(len(documents_2021_ids),len(documents_2021_contents))

old_id      object
contents    object
dtype: object
33401 33401


In [79]:
# storing the whole result in a csv file dataframe for k=250, 500, 786, 1000, 1375, 1500.
docs_id_list = []
topics_id_list = []
rel_id_list = []
content_list = []

for t_, docl_ in tp_rel1_dict.items():
    if len(docl_) > 0 :
        for di_ in docl_:
            if str(di_) in list(documents_2021['old_id']):
                docs_id_list.append(di_)
                topics_id_list.append(t_)
                rel_id_list.append(int(1))
                for x_, y_ in zip(documents_2021_ids, documents_2021_contents):
                    if str(di_) == str(x_):
                        content_list.append(y_)
                        break

for t_, docl_ in tp_rel2_dict.items():
    if len(docl_) > 0 :
        for di_ in docl_:
            if str(di_) in list(documents_2021['old_id']):
                docs_id_list.append(di_)
                topics_id_list.append(t_)
                rel_id_list.append(int(2))
                for x_, y_ in zip(documents_2021_ids, documents_2021_contents):
                    if str(di_) == str(x_):
                        content_list.append(y_)
                        break

In [80]:
len(content_list)

59

In [81]:
missed = pd.DataFrame(list(zip(docs_id_list, topics_id_list, rel_id_list, content_list)),
               columns =['doc_id', 'qid', 'rel', 'content'])
missed.to_csv("/content/drive/MyDrive/touche-2022-prototyping/missed_1500.csv", index=False)